# Glue Job Basic

#### 이번 Lab에서는 SageMaker notebook을 사용하여 간단하게 데이터를 읽고, ETL하고, Write하는 과정을 살펴봅니다.

#### S3에 업로드한 데이터를 읽어오기 위해 각자 S3 bucket에 지정한 account-id를 account_id 변수에 할당합니다.

In [ ]:
ACCOUNT_ID = ''
S3_BUCKET = 's3://aws-glue-hol-' + ACCOUNT_ID

#### Jupyter Notebook에서는 SparkContext(sc)와 SparkSession(spark)이 지원되므로 선언해서 사용하지 않아도 됩니다.

In [ ]:
sc

In [ ]:
spark

#### Jupyter Notebook에서는 아래와 같이 이미 선언된 SparkContext(sc)를 사용해서 GlueContext 생성이 가능합니다.

In [ ]:
from awsglue.context import GlueContext
from pyspark.sql.functions import regexp_extract, col

# Jupyter Notebook에서 이미 생성한 SparkContext(sc)를 사용해서 GlueContext 변수 생성
glueContext = GlueContext(sc)

#### S3 데이터를 읽어 DynamicFrame 생성합니다.

In [ ]:
titanic_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type = 's3',
    connection_options = {'paths': [S3_BUCKET + '/train']},
    format='csv',
    format_options={
        'withHeader': True,
        'delimiter': ','
    })

#### DynamicFrame을 DataFrame으로 변환하고 정상적으로 변환되었는지 확인합니다.

In [ ]:
titanic_csv_df = titanic_dyf.toDF()
titanic_csv_df.show()

#### 전체 데이터 수를 확인합니다.

In [ ]:
titanic_csv_df.count()

#### Spark API를 이용하여 Name 컬럼에서 유의미한 String만 추출하여 initial 컬럼을 생성합니다.

In [ ]:
titanic_csv_df = titanic_csv_df.withColumn('initial', regexp_extract(col('Name'), "(\w+)\.", 1))
titanic_csv_df.show()

#### Glue API를 이용하여 사용하지 않는 Name 컬럼은 삭제합니다.

In [ ]:
from awsglue.dynamicframe import DynamicFrame

titanic_csv_dyf = DynamicFrame.fromDF(titanic_csv_df, glueContext, 'titanic_csv_dyf').drop_fields('Name')
titanic_csv_dyf.toDF().show()

#### Json format으로 output 디렉토리에 저장합니다.

In [ ]:
titanic_csv_dyf.toDF().write \
    .format('json') \
    .mode('overwrite') \
    .save(S3_BUCKET + '/output')

#### Json으로 변환해서 저장된 데이터가 정상적으로 저장되었는지 확인합니다.

In [ ]:
titanic_json_df = glueContext.create_dynamic_frame_from_options(
    connection_type = 's3',
    connection_options = {'paths': [S3_BUCKET + '/output']},
    format='json').toDF()

In [ ]:
titanic_json_df.count()

In [ ]:
titanic_json_df.show()